<a href="https://colab.research.google.com/github/alkishc/publ/blob/main/atp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reset -f
import pandas as pd
import re
from io import StringIO
from google.colab import files
uploaded = files.upload()
import bokeh.io
import bokeh.plotting
import codecs

#uploaded = files.upload()
d=list(uploaded.keys())[0]
import matplotlib.pyplot as plt
import math
from bokeh.models.glyph import XYGlyph
from bokeh.plotting import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
output_notebook()
outputs=d+'_Figures.html'
bokeh.io.output_file(outputs)
LLCD=float(input("Enter LLC Distance=")) 
#@title File import and basic data { run: "auto" }

Saving R_1000_3sc.lis to R_1000_3sc.lis
Enter LLC Distance=400


In [2]:
#@title Data separation  { run: "auto" }
with codecs.open(d, encoding='utf-8',errors='ignore') as file:
    data = file.read()

Phasor=re.search('phasor switch currents.(.*)Solution at nodes with', data,re.DOTALL)
Branch= re.search('K to M   ======(.*)Memory storage', data,re.DOTALL)
Node=re.search('outputs follow.(.*)Selective branch outputs follow', data,re.DOTALL)

dfa = pd.read_csv(StringIO(Branch.group(1)), sep="\s+",skiprows=1)
dfb = pd.read_csv(StringIO(Node.group(1)), sep="\s+",skiprows=2)
dfc = pd.read_csv(StringIO(Phasor.group(1)), sep="\s+",skiprows=1)

dbr1 = dfa.dropna(axis=1) #Branch dataframe final
dnd1 = dfb.dropna(axis=1) #phasors dataframe finalfinalfinalfinalfinalfinalfinalfinal
dph1 = dfc.dropna(axis=1) #Nodes dataframe final

dbr1.columns = ['Bus_K', 'Bus_M', 'Magnitude', 'Degrees','Real_part' ,'Imag_part']
dnd1.columns = ['Bus_Name', 'Magnitude', 'Degrees','Real_part' ,'Imag_part']

branch_columns = dbr1[['Bus_K', 'Bus_M','Magnitude', 'Degrees']]
node_columns = dnd1[['Bus_Name','Magnitude', 'Degrees']]
phase_columns = dph1[['Node-K', 'Node-M','I-magn', 'Degrees']]

dbranch= branch_columns.copy()
dnode=node_columns.copy()
dphase1=phase_columns.copy()
dphase=dphase1.rename(columns={"Node-K": "NodeK", "Node-M": "NodeM","I-magn": "Imagn"})
out=d+'_Data.xlsx'
with pd.ExcelWriter(out) as writer:
    dphase.to_excel(writer, sheet_name='Currents', index=False,header=False)
    dnode.to_excel(writer, sheet_name='VNode', index=False,header=False)
    dbranch.to_excel(writer, sheet_name='VBranch', index=False,header=False)

q=input("save .xlsx file? (y/n)")
if q==str("y"):
    files.download(out)     
    print('starting downloading',"{}".format(d))
else:
  print("Continue with next for plotting")
  pass

save .xlsx file? (y/n)n
Continue with next for plotting


In [ ]:
import warnings
from bokeh.layouts import row
from bokeh.layouts import gridplot
from bokeh.models import BasicTickFormatter
warnings.filterwarnings('ignore')

#@title Conductor and sheath plots { run: "auto" }
Save = False #@param {type:"boolean"}

CondA=dnode[dnode.Bus_Name.str.contains('R(.*)A')] 
CondA.reset_index(inplace=True)
CondA['Distance']=CondA.index*LLCD
CondA['Voltage_RMS']=CondA.Magnitude/math.sqrt(2)

CondB=dnode[dnode.Bus_Name.str.contains('R(.*)B')] 
CondB.reset_index(inplace=True)
CondB['Distance']=CondB.index*LLCD
CondB['Voltage_RMS']=CondB.Magnitude/math.sqrt(2)

CondC=dnode[dnode.Bus_Name.str.contains('R(.*)C')] 
CondC.reset_index(inplace=True)
CondC['Distance']=CondC.index*LLCD
CondC['Voltage_RMS']=CondC.Magnitude/math.sqrt(2)

SheathA=dnode[dnode.Bus_Name.str.contains('F(.*)A')] 
SheathA.reset_index(inplace=True)
SheathA['Distance']=SheathA.index*LLCD
SheathA['Voltage_RMS']=SheathA.Magnitude/math.sqrt(2)

SheathB=dnode[dnode.Bus_Name.str.contains('F(.*)B')] 
SheathB.reset_index(inplace=True)
SheathB['Distance']=SheathB.index*LLCD
SheathB['Voltage_RMS']=SheathB.Magnitude/math.sqrt(2)

SheathC=dnode[dnode.Bus_Name.str.contains('F(.*)C')] 
SheathC.reset_index(inplace=True)
SheathC['Distance']=SheathC.index*LLCD
SheathC['Voltage_RMS']=SheathC.Magnitude/math.sqrt(2)

NodePA=dnode[dnode.Bus_Name.str.contains('P(.*)A')] 
NodePA.reset_index(inplace=True)
NodePA['Distance']=NodePA.index*LLCD
NodePA['Voltage_RMS']=NodePA.Magnitude/math.sqrt(2)

NodePB=dnode[dnode.Bus_Name.str.contains('P(.*)B')] 
NodePB.reset_index(inplace=True)
NodePB['Distance']=NodePB.index*LLCD
NodePB['Voltage_RMS']=NodePB.Magnitude/math.sqrt(2)

NodePC=dnode[dnode.Bus_Name.str.contains('P(.*)C')] 
NodePC.reset_index(inplace=True)
NodePC['Distance']=NodePC.index*LLCD
NodePC['Voltage_RMS']=NodePC.Magnitude/math.sqrt(2)


figCond = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title= "Conductor Voltage RMS")
figCond.line(CondA['Distance'],CondA['Voltage_RMS'],legend_label="A:", color='red',alpha=0.8)
figCond.line(CondB['Distance'],CondB['Voltage_RMS'],legend_label="B:", color='green',alpha=0.8)
figCond.line(CondC['Distance'],CondC['Voltage_RMS'],legend_label="C:", color='blue',alpha=0.8)
figCond.legend.click_policy="hide"
figCond.legend.background_fill_alpha = 0.0


figSheath = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title=" Sheath Voltage RMS")
figSheath.line(SheathA['Distance'],SheathA['Voltage_RMS'],legend_label="A:", color='red',alpha=0.8)
figSheath.line(SheathB['Distance'],SheathB['Voltage_RMS'],legend_label="B:", color='green',alpha=0.8)
figSheath.line(SheathC['Distance'],SheathC['Voltage_RMS'],legend_label="C:", color='blue',alpha=0.8)
figSheath.legend.click_policy="hide"
figSheath.legend.background_fill_alpha = 0.0


figNode2 = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title=" Common armour & FOA & FOT")
figNode2.line(NodePA['Distance'],NodePA['Voltage_RMS'],legend_label="FOT: Max="+str(round(max(NodePA.Voltage_RMS),1))+"  V", color='red',alpha=0.8)
figNode2.line(NodePB['Distance'],NodePB['Voltage_RMS'],legend_label="FOA: Max="+str(round(max(NodePB.Voltage_RMS),1))+"  V", color='blue',alpha=0.8)
figNode2.line(NodePC['Distance'],NodePC['Voltage_RMS'],legend_label="Common Armour: Max="+str(round(max(NodePC.Voltage_RMS),1))+"  V", color='green',alpha=0.8)
figNode2.legend.click_policy="hide"
figNode2.legend.background_fill_alpha = 0.0



brA=dbranch[dbranch.Bus_K.str.contains('S(.*)A')] 
brA.reset_index(inplace=True)
brA['Distance']=brA.index*LLCD
brA['Voltage_RMS']=brA.Magnitude/math.sqrt(2)

brB=dbranch[dbranch.Bus_K.str.contains('S(.*)B')] 
brB.reset_index(inplace=True)
brB['Distance']=brB.index*LLCD
brB['Voltage_RMS']=brB.Magnitude/math.sqrt(2)

brC=dbranch[dbranch.Bus_K.str.contains('S(.*)C')] 
brC.reset_index(inplace=True)
brC['Distance']=brC.index*LLCD
brC['Voltage_RMS']=brC.Magnitude/math.sqrt(2)

brA1=dbranch[dbranch.Bus_K.str.contains('F(.*)A')] 
brA=brA1[brA1.Bus_M.str.contains('F(.*)')] 
brA.reset_index(inplace=True)
brA['Distance']=brA.index*LLCD
brA['Voltage_RMS']=brA.Magnitude/math.sqrt(2)

brB1=dbranch[dbranch.Bus_K.str.contains('F(.*)B')] 
brB=brB1[brB1.Bus_M.str.contains('F(.*)')] 
brB.reset_index(inplace=True)
brB['Distance']=brB.index*LLCD
brB['Voltage_RMS']=brB.Magnitude/math.sqrt(2)

brC1=dbranch[dbranch.Bus_K.str.contains('F(.*)C')] 
brC=brC1[brC1.Bus_M.str.contains('F(.*)A')]
brC.reset_index(inplace=True)
brC['Distance']=brC.index*LLCD
brC['Voltage_RMS']=brC.Magnitude/math.sqrt(2)

figbr = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title="Screen to foil Voltage RMS")
figbr.line(brA['Distance'],brA['Voltage_RMS'],legend_label="Phase= A: Max="+str(round(max(brA.Voltage_RMS),4))+"  V , Min="+str(round(min(brA.Voltage_RMS),1))+"  V", color='red',alpha=0.8)
figbr.line(brB['Distance'],brB['Voltage_RMS'],legend_label="Phase= B: Max="+str(round(max(brB.Voltage_RMS),4))+"  V , Min="+str(round(min(brB.Voltage_RMS),1))+"  V", color='green',alpha=0.8)
figbr.line(brC['Distance'],brC['Voltage_RMS'],legend_label="Phase= C: Max="+str(round(max(brC.Voltage_RMS),4))+"  V , Min="+str(round(min(brC.Voltage_RMS),1))+"  V", color='blue',alpha=0.8)
figbr.legend.click_policy="hide"
figbr.legend.background_fill_alpha = 0.0


figbrP = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title="Foil to Foil - Voltage RMS")
figbrP.line(brA['Distance'],brA['Voltage_RMS'],legend_label="AB: Max="+str(round(max(brA.Voltage_RMS),4))+"  V", color='red',alpha=0.8)
figbrP.line(brB['Distance'],brB['Voltage_RMS'],legend_label="BC: Max="+str(round(max(brB.Voltage_RMS),4))+"  V", color='green',alpha=0.8)
figbrP.line(brC['Distance'],brC['Voltage_RMS'],legend_label="CA: Max="+str(round(max(brC.Voltage_RMS),4))+"  V", color='blue',alpha=0.8)
figbrP.legend.click_policy="hide"
figbrP.legend.background_fill_alpha = 0.0


#figCond.xaxis[0].formatter = NumeralTickFormatter(format=".0%")
#figCond.yaxis.formatter = NumeralTickFormatter(format='use_scientific')
figCond.yaxis.formatter = BasicTickFormatter(use_scientific=True)

grid = gridplot([figCond, figSheath, figNode2,figbr,figbrP],ncols=3)

show(grid)

#q=input('Save page? (y/n)')
if Save==True:
    files.download(outputs)     
    print('starting downloading',"{}".format(d))
else:
  print("Continue with next for plotting")
  pass

In [ ]:
#@title Second Plot

import warnings
from bokeh.layouts import row
from bokeh.layouts import gridplot
from bokeh.models import BasicTickFormatter
warnings.filterwarnings('ignore')

CondA=dnode[dnode.Bus_Name.str.contains('S(.*)A')] 
CondA.reset_index(inplace=True)
CondA['Distance']=CondA.index*LLCD
CondA['Voltage_RMS']=CondA.Magnitude/math.sqrt(2)

CondB=dnode[dnode.Bus_Name.str.contains('S(.*)B')] 
CondB.reset_index(inplace=True)
CondB['Distance']=CondB.index*LLCD
CondB['Voltage_RMS']=CondB.Magnitude/math.sqrt(2)

CondC=dnode[dnode.Bus_Name.str.contains('S(.*)C')] 
CondC.reset_index(inplace=True)
CondC['Distance']=CondC.index*LLCD
CondC['Voltage_RMS']=CondC.Magnitude/math.sqrt(2)

SheathA=dnode[dnode.Bus_Name.str.contains('F(.*)A')] 
SheathA.reset_index(inplace=True)
SheathA['Distance']=SheathA.index*LLCD
SheathA['Voltage_RMS']=SheathA.Magnitude/math.sqrt(2)

SheathB=dnode[dnode.Bus_Name.str.contains('F(.*)B')] 
SheathB.reset_index(inplace=True)
SheathB['Distance']=SheathB.index*LLCD
SheathB['Voltage_RMS']=SheathB.Magnitude/math.sqrt(2)

SheathC=dnode[dnode.Bus_Name.str.contains('F(.*)C')] 
SheathC.reset_index(inplace=True)
SheathC['Distance']=SheathC.index*LLCD
SheathC['Voltage_RMS']=SheathC.Magnitude/math.sqrt(2)

NodePA=dnode[dnode.Bus_Name.str.contains('P(.*)A')] 
NodePA.reset_index(inplace=True)
NodePA['Distance']=NodePA.index*LLCD
NodePA['Voltage_RMS']=NodePA.Magnitude/math.sqrt(2)

NodePB=dnode[dnode.Bus_Name.str.contains('P(.*)B')] 
NodePB.reset_index(inplace=True)
NodePB['Distance']=NodePB.index*LLCD
NodePB['Voltage_RMS']=NodePB.Magnitude/math.sqrt(2)

NodePC=dnode[dnode.Bus_Name.str.contains('P(.*)C')] 
NodePC.reset_index(inplace=True)
NodePC['Distance']=NodePC.index*LLCD
NodePC['Voltage_RMS']=NodePC.Magnitude/math.sqrt(2)


figCond = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title= "Conductor Voltage RMS")
figCond.line(CondA['Distance'],CondA['Voltage_RMS'],legend_label="A:", color='red',alpha=0.8)
figCond.line(CondB['Distance'],CondB['Voltage_RMS'],legend_label="B:", color='green',alpha=0.8)
figCond.line(CondC['Distance'],CondC['Voltage_RMS'],legend_label="C:", color='blue',alpha=0.8)
figCond.legend.click_policy="hide"
figCond.legend.background_fill_alpha = 0.0


figSheath = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title=" Sheath Voltage RMS")
figSheath.line(SheathA['Distance'],SheathA['Voltage_RMS'],legend_label="A:", color='red',alpha=0.8)
figSheath.line(SheathB['Distance'],SheathB['Voltage_RMS'],legend_label="B:", color='green',alpha=0.8)
figSheath.line(SheathC['Distance'],SheathC['Voltage_RMS'],legend_label="C:", color='blue',alpha=0.8)
figSheath.legend.click_policy="hide"
figSheath.legend.background_fill_alpha = 0.0


figNode2 = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title=" Common armour & FOA & FOT")
figNode2.line(NodePA['Distance'],NodePA['Voltage_RMS'],legend_label="FOT: Max="+str(round(max(NodePA.Voltage_RMS),1))+"  V", color='red',alpha=0.8)
figNode2.line(NodePB['Distance'],NodePB['Voltage_RMS'],legend_label="FOA: Max="+str(round(max(NodePB.Voltage_RMS),1))+"  V", color='blue',alpha=0.8)
figNode2.line(NodePC['Distance'],NodePC['Voltage_RMS'],legend_label="Common Armour: Max="+str(round(max(NodePC.Voltage_RMS),1))+"  V", color='green',alpha=0.8)
figNode2.legend.click_policy="hide"
figNode2.legend.background_fill_alpha = 0.0



brA=dbranch[dbranch.Bus_K.str.contains('S(.*)A')] 
brA.reset_index(inplace=True)
brA['Distance']=brA.index*LLCD
brA['Voltage_RMS']=brA.Magnitude/math.sqrt(2)

brB=dbranch[dbranch.Bus_K.str.contains('S(.*)B')] 
brB.reset_index(inplace=True)
brB['Distance']=brB.index*LLCD
brB['Voltage_RMS']=brB.Magnitude/math.sqrt(2)

brC=dbranch[dbranch.Bus_K.str.contains('S(.*)C')] 
brC.reset_index(inplace=True)
brC['Distance']=brC.index*LLCD
brC['Voltage_RMS']=brC.Magnitude/math.sqrt(2)

brA1=dbranch[dbranch.Bus_K.str.contains('F(.*)A')] 
brA=brA1[brA1.Bus_M.str.contains('F(.*)')] 
brA.reset_index(inplace=True)
brA['Distance']=brA.index*LLCD
brA['Voltage_RMS']=brA.Magnitude/math.sqrt(2)

brB1=dbranch[dbranch.Bus_K.str.contains('F(.*)B')] 
brB=brB1[brB1.Bus_M.str.contains('F(.*)')] 
brB.reset_index(inplace=True)
brB['Distance']=brB.index*LLCD
brB['Voltage_RMS']=brB.Magnitude/math.sqrt(2)

brC1=dbranch[dbranch.Bus_K.str.contains('F(.*)C')] 
brC=brC1[brC1.Bus_M.str.contains('F(.*)A')]
brC.reset_index(inplace=True)
brC['Distance']=brC.index*LLCD
brC['Voltage_RMS']=brC.Magnitude/math.sqrt(2)

figbr = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title="Screen to foil Voltage RMS")
figbr.line(brA['Distance'],brA['Voltage_RMS'],legend_label="Phase= A: Max="+str(round(max(brA.Voltage_RMS),4))+"  V , Min="+str(round(min(brA.Voltage_RMS),1))+"  V", color='red',alpha=0.8)
figbr.line(brB['Distance'],brB['Voltage_RMS'],legend_label="Phase= B: Max="+str(round(max(brB.Voltage_RMS),4))+"  V , Min="+str(round(min(brB.Voltage_RMS),1))+"  V", color='green',alpha=0.8)
figbr.line(brC['Distance'],brC['Voltage_RMS'],legend_label="Phase= C: Max="+str(round(max(brC.Voltage_RMS),4))+"  V , Min="+str(round(min(brC.Voltage_RMS),1))+"  V", color='blue',alpha=0.8)
figbr.legend.click_policy="hide"
figbr.legend.background_fill_alpha = 0.0


figbrP = figure(plot_height=400,plot_width=450, y_axis_label='RMS Magnitude', x_axis_label='Distance [m]', title="Foil to Foil - Voltage RMS")
figbrP.line(brA['Distance'],brA['Voltage_RMS'],legend_label="AB: Max="+str(round(max(brA.Voltage_RMS),4))+"  V", color='red',alpha=0.8)
figbrP.line(brB['Distance'],brB['Voltage_RMS'],legend_label="BC: Max="+str(round(max(brB.Voltage_RMS),4))+"  V", color='green',alpha=0.8)
figbrP.line(brC['Distance'],brC['Voltage_RMS'],legend_label="CA: Max="+str(round(max(brC.Voltage_RMS),4))+"  V", color='blue',alpha=0.8)
figbrP.legend.click_policy="hide"
figbrP.legend.background_fill_alpha = 0.0


#figCond.xaxis[0].formatter = NumeralTickFormatter(format=".0%")
#figCond.yaxis.formatter = NumeralTickFormatter(format='use_scientific')
figCond.yaxis.formatter = BasicTickFormatter(use_scientific=True)

grid = gridplot([figCond, figSheath, figNode2,figbr,figbrP],ncols=3)

show(grid)

In [ ]:
dbranch